# 0. ETF Selection

We select the SPDR Gold Shares (GLD) ETF as the gold ETF. It is traded on Nasdaq, the currency is USD.

Similarly, we choose the Amundi CAC 40 UCITS ETF-C (C40.PA) as the equity ETF. It will track the CAC 40 index of France. It is traded on Paris Euronext, the currency is EUR.

The currency for Bitcoin is USD.

Data source: https://finance.yahoo.com/

# 1. Data Importing

In [114]:
import arch
import holidays
import pmdarima
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import VECM
%matplotlib inline

In [115]:
gold_df = pd.read_csv("data/SPDR Gold Shares (GLD) Jan - Dec 2020.csv")
equity_df = pd.read_csv("data/Amundi CAC 40 UCITS ETF-C (C40.PA) Jan 2020 - Dec 2020.csv")
bitcoin_df = pd.read_csv('data/Bitcoin USD (BTC-USD) Jan 2020 - Dec 2020.csv')

Convert the data into the datetime format and make it the index to query the dataframe easier.

In [116]:
def convert_df(df):
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.set_index("Date", inplace=True)   
    return df

gold_df = convert_df(gold_df)
equity_df = convert_df(equity_df)
bitcoin_df = convert_df(bitcoin_df)


We use the common subset of days for all 3 time series to make them comparable and to run the test for cointegration in question 7.

In [117]:
missing = set(equity_df.index) - set(gold_df.index)

In [118]:
def remove_missing_days(df, indices, missing):
    return df.loc[[index for index in indices if index not in missing]]

In [119]:
gold_df = remove_missing_days(gold_df, equity_df.index, missing)
equity_df = remove_missing_days(equity_df, equity_df.index, missing)
bitcoin_df = remove_missing_days(bitcoin_df, equity_df.index, missing)

In [120]:
assert equity_df.shape == gold_df.shape
assert equity_df.shape == bitcoin_df.shape

We then fill some missing values in the equity series with its old values to avoid data leakage.

In [121]:
equity_df = equity_df.ffill()

Verify that the time range is correct.

In [122]:
gold_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,143.860001,144.210007,143.399994,143.949997,143.949997,7733800
2020-01-03,145.750000,146.320007,145.399994,145.860001,145.860001,12272800
2020-01-06,148.440002,148.479996,146.949997,147.389999,147.389999,14403300
2020-01-07,147.570007,148.139999,147.429993,147.970001,147.970001,7978500
2020-01-08,148.490005,148.610001,146.139999,146.860001,146.860001,22248500


In [123]:
gold_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,175.100006,176.210007,175.059998,175.649994,175.649994,6542800
2020-12-24,175.550003,176.369995,175.509995,176.350006,176.350006,3695400
2020-12-28,177.259995,177.910004,175.630005,175.710007,175.710007,7778700
2020-12-29,176.250000,176.970001,175.570007,176.350006,176.350006,5983700
2020-12-30,176.440002,177.720001,176.440002,177.699997,177.699997,5914000


In [124]:
equity_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,88.589996,89.239998,88.589996,89.239998,89.239998,124.0
2020-01-03,88.489998,88.889999,88.309998,88.769997,88.769997,0.0
2020-01-06,88.529999,88.599998,87.750000,88.559998,88.559998,563.0
2020-01-07,88.839996,89.260002,88.500000,88.500000,88.500000,0.0
2020-01-08,88.120003,89.300003,88.120003,89.139999,89.139999,212.0


In [125]:
equity_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,82.480003,83.330002,82.000000,83.250000,83.250000,68117.0
2020-12-24,83.419998,83.550003,82.930000,83.139999,83.139999,32892.0
2020-12-28,83.849998,84.290001,83.410004,84.160004,84.160004,20640.0
2020-12-29,84.440002,84.680000,84.320000,84.449997,84.449997,20668.0
2020-12-30,84.489998,84.699997,84.300003,84.339996,84.339996,3829.0


In [126]:
bitcoin_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481031
2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,7769.219238,23276261598
2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,8163.692383,28767291326
2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,8079.862793,31672559264


In [127]:
bitcoin_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,23781.974609,24024.490234,22802.646484,23241.345703,23241.345703,51146161904
2020-12-24,23240.203125,23768.337891,22777.597656,23735.949219,23735.949219,41080759712
2020-12-28,26280.822266,27389.111328,26207.640625,27084.808594,27084.808594,49056742892
2020-12-29,27081.810547,27370.720703,25987.298828,27362.437500,27362.437500,45265946774
2020-12-30,27360.089844,28937.740234,27360.089844,28840.953125,28840.953125,51287442703


# 2. Data Processing

We use adjusted close prices to calculate the daily returns. Adjusted close prices are the prices that already take into account stock split and dividends, which reflex more accurate the change of the prices.

In [128]:
equity_df["Daily Return"] = equity_df["Adj Close"].pct_change(1)
equity_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,88.589996,89.239998,88.589996,89.239998,89.239998,124.0,NaN
2020-01-03,88.489998,88.889999,88.309998,88.769997,88.769997,0.0,-0.005267
2020-01-06,88.529999,88.599998,87.750000,88.559998,88.559998,563.0,-0.002366
2020-01-07,88.839996,89.260002,88.500000,88.500000,88.500000,0.0,-0.000677
2020-01-08,88.120003,89.300003,88.120003,89.139999,89.139999,212.0,0.007232


In [129]:
gold_df["Daily Return"] = gold_df["Adj Close"].pct_change(1)
gold_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,143.860001,144.210007,143.399994,143.949997,143.949997,7733800,NaN
2020-01-03,145.750000,146.320007,145.399994,145.860001,145.860001,12272800,0.013269
2020-01-06,148.440002,148.479996,146.949997,147.389999,147.389999,14403300,0.010489
2020-01-07,147.570007,148.139999,147.429993,147.970001,147.970001,7978500,0.003935
2020-01-08,148.490005,148.610001,146.139999,146.860001,146.860001,22248500,-0.007502


In [130]:
bitcoin_df["Daily Return"] = bitcoin_df["Adj Close"].pct_change(1)
bitcoin_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,NaN
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481031,0.051452
2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,7769.219238,23276261598,0.057773
2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,8163.692383,28767291326,0.050774
2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,8079.862793,31672559264,-0.010269


# 5. Category 1 Models: Just use 1 variable.

In [131]:
# Helper
df_names = {0: "gold ETF", 1: "equity ETF", 2: "Bitcoin"}
dfs = [gold_df, equity_df, bitcoin_df]
def get_data(df, month_start, month_end, column=None):
    data = df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")]
    if column: 
        data = data[column]
    return data

In [132]:
def summarize_data(df):
    data = get_data(df, 3, 12, "Adj Close")
    moving_avg = data.rolling(20, min_periods=1).mean()
    return get_data(moving_avg, 4, 12)


In [133]:
def get_data(df, month_start, month_end, column):
    return df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")][column]

In [134]:
def fit_arima(data, exog= None):
    model = ARIMA(data, exog=exog, order=(2,0,2))
    model_fit = model.fit()
    return model_fit

In [135]:
def fit_garch(data, garch_type="GARCH"):
    if garch_type == "TARCH":
        garch = arch.arch_model(data, vol='TGARCH', p=1, o=1, q=1, power=1)
    else:
        garch = arch.arch_model(data, vol=garch_type, p=1, o=0, q=1)
    garch_fitted = garch.fit()
    print(garch_fitted.summary())
    if garch_type != "FIGARCH":
        omega = garch_fitted.params["omega"]
        alpha = garch_fitted.params["alpha[1]"]
        beta = garch_fitted.params["beta[1]"]
        print(f"Unconditional variance: {omega/(1 - alpha - beta)}")
    return garch_fitted

In [136]:
bitcoin_q2 = get_data(bitcoin_df, 4, 6, "Daily Return")
bitcoin_q3 = get_data(bitcoin_df, 7, 9, "Daily Return")
bitcoin_q4 = get_data(bitcoin_df, 10, 12, "Daily Return")

In [137]:
equity_q2 = get_data(equity_df, 4, 6, "Daily Return")
equity_q3 = get_data(equity_df, 7, 9, "Daily Return")
equity_q4 = get_data(equity_df, 10, 12, "Daily Return")

In [138]:
gold_q2 = get_data(gold_df, 4, 6, "Daily Return")
gold_q3 = get_data(gold_df, 7, 9, "Daily Return")
gold_q4 = get_data(gold_df, 10, 12, "Daily Return")

In [139]:
bitcoin_q2.shape

(39,)

In [140]:
bitcoin_q3.shape

(43,)

In [141]:
bitcoin_q4.shape

(42,)

In [142]:
gold_q2.shape

(39,)

In [143]:
gold_q3.shape

(43,)

Our first model with ARIMA is a simple one, we'll allocate our capital totally to bitcoin if we predict that the next day price will rise, otherwise we will short 100%.

The total return for Q3 is 3.3%, not very impressive, but for Q4, it's 82%, which is much better. Return for 2 quarters is 87%

In [144]:
model = fit_arima(bitcoin_q2)
return_pred = model.predict(start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.03305217361367663

In [145]:
model = fit_arima(bitcoin_q3)
return_pred = model.predict(start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred >0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at 

0.8192927891621655

In [146]:
(1 + return_q3) * (1 + return_q4) - 1

0.8794243702836635

# 6. Category 2 Models: Just use 2 variables

With VARMA model, we use the equity returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 28% while for Q4, it's 63%.  Return for 2 quarters is 108%

In [147]:
model = fit_arima(bitcoin_q2, equity_q2)
return_pred = model.predict(exog=equity_q3, start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.2782697582897862

In [148]:
model = fit_arima(bitcoin_q3, equity_q3)
return_pred = model.predict(exog=equity_q4, start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at 

0.6289117237103818

In [149]:
(1 + return_q3) * (1 + return_q4) - 1

1.0821885953426689

# 7 . Category 3 Models: Use all 3 variables

With VARMA model, we use both the equity and gold returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 50% while for Q4, it's 40%. Return for 2 quarters is 109%

In [150]:
model = fit_arima(bitcoin_q2, pd.concat([equity_q2, gold_q2], axis=1))
return_pred = model.predict(exog=pd.concat([equity_q3, gold_q3], axis=1), start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.4981117995367046

In [151]:
model = fit_arima(bitcoin_q3, pd.concat([equity_q3, gold_q3], axis=1))
return_pred = model.predict(exog=pd.concat([equity_q4, gold_q4], axis=1), start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.3971697263227958

In [152]:
(1 + return_q3) * (1 + return_q4) - 1

1.0931164529596487

In [167]:
index = 2
df = dfs[index]
print(f"{model} model for {df_names[index]} from April to December")
data = get_data(df, 4, 6, "Daily Return")
data = data.dropna()
garch_fitted = fit_garch(data, "GARCH")
forecasts = garch_fitted.forecast(horizon=43)

<statsmodels.tsa.vector_ar.vecm.VECM object at 0x0000025A65E5DCA0> model for Bitcoin from April to December
Iteration:      1,   Func. Count:      6,   Neg. LLF: 118.77728750503083
Iteration:      2,   Func. Count:     15,   Neg. LLF: 28.011802178610246
Iteration:      3,   Func. Count:     23,   Neg. LLF: -65.99105261555415
Iteration:      4,   Func. Count:     29,   Neg. LLF: -66.08154265855389
Iteration:      5,   Func. Count:     35,   Neg. LLF: -66.11790569785661
Iteration:      6,   Func. Count:     41,   Neg. LLF: -66.14856812230553
Iteration:      7,   Func. Count:     47,   Neg. LLF: -66.30893104997838
Iteration:      8,   Func. Count:     53,   Neg. LLF: 1672122038.5633073
Iteration:      9,   Func. Count:     62,   Neg. LLF: 24566545.909773417
Iteration:     10,   Func. Count:     70,   Neg. LLF: -66.5149996812599
Iteration:     11,   Func. Count:     75,   Neg. LLF: -66.55468093206487
Iteration:     12,   Func. Count:     80,   Neg. LLF: -66.50010091457224
Iteration:     13

C:\Users\trinh\AppData\Roaming\Python\Python38\site-packages\arch\univariate\base.py:292: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.00194. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [169]:
forecasts.mean

,h.01,h.02,h.03,h.04,h.05,h.06,h.07,h.08,h.09,h.10,...,h.34,h.35,h.36,h.37,h.38,h.39,h.40,h.41,h.42,h.43
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Compare models using cumulative return and volatility

# 7. Modelling Cointegration

In [156]:
data1 = get_data(gold_df, 4, 6, "Daily Return").values
data2 = get_data(equity_df, 4, 6, "Daily Return").values
data3 = get_data(bitcoin_df, 4, 6, "Daily Return").values

In [157]:
coint(data1, np.array([data2, data3]).T)

(-6.076460637243138,
 5.692753720973776e-06,
 array([-4.69574274, -3.97301265, -3.618289  ]))

In [158]:
coint(data2, np.array([data1, data3]).T)

(-6.203463603398757,
 2.972096825666223e-06,
 array([-4.69574274, -3.97301265, -3.618289  ]))

In [159]:
coint(data3, np.array([data1, data2]).T)

(-5.93178147713331,
 1.1733043376457846e-05,
 array([-4.69574274, -3.97301265, -3.618289  ]))

p-value < 0.01 for all tests, we conclude that there are cointegrating vectors.

Vector Error Correction Model

In [160]:
data = np.array([data1, data2, data3]).T
model = VECM(data, coint_rank=1)
vecm_res = model.fit()
vecm_res.gamma.round(4)
vecm_res.summary()
forecast, lower, upper = vecm_res.predict(5, 0.05)
print("lower bounds of confidence intervals:")
print(lower.round(3))
print("\npoint forecasts:")
print(forecast.round(3))
print("\nupper bounds of confidence intervals:")
print(upper.round(3))

lower bounds of confidence intervals:
[[-0.018 -0.046 -0.086]
 [-0.024 -0.046 -0.122]
 [-0.03  -0.053 -0.142]
 [-0.034 -0.058 -0.158]
 [-0.038 -0.061 -0.172]]

point forecasts:
[[ 0.01  -0.004  0.021]
 [ 0.008  0.     0.002]
 [ 0.008 -0.001  0.007]
 [ 0.008 -0.003  0.007]
 [ 0.008 -0.002  0.008]]

upper bounds of confidence intervals:
[[0.037 0.039 0.128]
 [0.039 0.046 0.126]
 [0.046 0.051 0.155]
 [0.05  0.052 0.171]
 [0.054 0.056 0.189]]


In [161]:
data1 = get_data(gold_df, 7, 9, "Daily Return").values
data2 = get_data(equity_df, 7, 9, "Daily Return").values
data3 = get_data(bitcoin_df, 7, 9, "Daily Return").values

In [162]:
coint(data1, np.array([data2, data3]).T)

(-6.922723638048469,
 5.848143640071725e-08,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [163]:
coint(data2, np.array([data1, data3]).T)

(-3.7317779390504793,
 0.05119188151529683,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [164]:
coint(data3, np.array([data1, data2]).T)

(-8.835667550986745,
 3.3294143920743777e-13,
 array([-4.6556178 , -3.95031996, -3.60224723]))

Interestingly, we can't reject the null hypothesis for the test for the equity ETF and the combintion of gold and bitcoin ETF. We can reject the null hypothesis for the other two tests.

In [165]:
data = np.array([data1, data2, data3]).T
model = VECM(data, coint_rank=1)
vecm_res = model.fit()
vecm_res.gamma.round(4)
vecm_res.summary()
forecast, lower, upper = vecm_res.predict(5, 0.05)
print("lower bounds of confidence intervals:")
print(lower.round(3))
print("\npoint forecasts:")
print(forecast.round(3))
print("\nupper bounds of confidence intervals:")
print(upper.round(3))

lower bounds of confidence intervals:
[[-0.022 -0.023 -0.05 ]
 [-0.03  -0.024 -0.065]
 [-0.035 -0.027 -0.074]
 [-0.041 -0.028 -0.085]
 [-0.045 -0.029 -0.092]]

point forecasts:
[[ 0.01  -0.001  0.018]
 [ 0.005 -0.001  0.007]
 [ 0.008 -0.002  0.015]
 [ 0.007 -0.002  0.01 ]
 [ 0.007 -0.002  0.014]]

upper bounds of confidence intervals:
[[0.042 0.022 0.085]
 [0.041 0.023 0.079]
 [0.051 0.023 0.105]
 [0.054 0.024 0.106]
 [0.059 0.025 0.119]]


In [174]:
res = vecm_res.predict(steps=43)
res

array([[ 0.00991084, -0.00056451,  0.01759424],
       [ 0.00542532, -0.00071332,  0.00730052],
       [ 0.0077745 , -0.00194259,  0.01548352],
       [ 0.00671912, -0.00207078,  0.01035303],
       [ 0.00715609, -0.00171956,  0.01367679],
       [ 0.00699781, -0.00188074,  0.01146103],
       [ 0.00702813, -0.00180125,  0.01288362],
       [ 0.00705039, -0.00186321,  0.01200524],
       [ 0.00701353, -0.00181825,  0.01253769],
       [ 0.00704883, -0.00184634,  0.01221922],
       [ 0.00702029, -0.00182935,  0.01240623],
       [ 0.00704146, -0.00183977,  0.0122987 ],
       [ 0.00702661, -0.00183345,  0.01235926],
       [ 0.00703662, -0.00183718,  0.0123259 ],
       [ 0.00703008, -0.00183503,  0.01234379],
       [ 0.00703424, -0.00183625,  0.01233451],
       [ 0.00703165, -0.00183557,  0.01233912],
       [ 0.00703323, -0.00183594,  0.01233697],
       [ 0.00703228, -0.00183575,  0.01233788],
       [ 0.00703284, -0.00183584,  0.01233757],
       [ 0.00703252, -0.0018358 ,  0.012